## Using the FINBOURNE Python SDKs

We provide many tools for interacting with FINBOURNE products in Python and Jupyter.

The FINBOURNE Python SDKs provides a set of objects and functions which enable you to call our APIs without having to write any direct http or serialisation code. We also provide other manually-written packages which make interacting with LUSID programmatically easier.
In this section we'll:
- Introduce the FINBOURNE Python SDKs, describing how they are used.
- Describe some of the tools for interacting with LUSID in our hosted Jupyter environment.

### Authentication
We have a set of SDKs such as the lusid-sdk-python, drive-sdk-python and finbourne-access-sdk-python, all hosted on Pypi. These SDKs provide api objects, with methods that can be used to perform actions in LUSID.

In [ ]:
!pip3 install -U lusid-sdk

All authenticated calls to the LUSID API require an OpenID Connect ID token which is issued from your token issuer url. The details of these can be found on your LUSID portal under "Applications" within the "Identity and Access Management" section.

This is explained in greater detail in our [knowledge base.](https://support.lusid.com/knowledgebase/article/KA-01667/en-us)

In [ ]:
import lusid
import os
from lusidjam import RefreshingToken
from lusid.extensions import (
    SyncApiClientFactory,
    ArgsConfigurationLoader,
    EnvironmentVariablesConfigurationLoader,
    SecretsFileConfigurationLoader
)

try:
    # Authenticate to SDK
    # Run the Notebook in Jupyterhub for your LUSID domain and authenticate automatically
    secrets_path = os.getenv("FBN_SECRETS_PATH")
    # Run the Notebook locally using a secrets file (see https://support.lusid.com/docs/how-do-i-use-an-api-access-token-with-the-lusid-sdk)
    if secrets_path is None:
        secrets_path = os.path.join(os.path.dirname(os.getcwd()), "secrets.json")
        print()

    # Initiate an API Factory which is the client side object for interacting with LUSID APIs
    config_loaders=[
        ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
        EnvironmentVariablesConfigurationLoader(),
        SecretsFileConfigurationLoader(secrets_path)]
    api_factory = SyncApiClientFactory(config_loaders=config_loaders)
    
    print([api for api in dir(lusid.api) if "Api" in api])
except Exception as e:
    print(e)



Here's an example using the LUSID Python sdk. We initialise an `api_factory` using our access details, which can be provided in either a secrets json file, or as environment variables. Here we have some environment variables set, we use `RefreshingToken` to grab a token using these environment variables.

We then print a list of the APIs available in the LUSID SDK.

In [ ]:
try:
    api_instance = api_factory.build(lusid.ApplicationMetadataApi)
    # GetLusidVersions: Get LUSID versions
    api_response = api_instance.get_lusid_versions()
    print(api_response)
except lusid.rest.ApiException as e:
    print("Exception when calling ApplicationMetadataApi->get_lusid_versions: %s\n" % e)


Here, we use our `api_factory` to build an `ApplicationMetadatApi` object, which we can use to send requests for our ApplicationMetadata REST endpoints.
We then call the `get_lusid_versions` method on this object, which sends a request to the ApplicationMetadataApi->get_lusid_versions endpoint, returning the result as a JSON string.

### Tools for interacting with LUSID in our hosted Jupyter environment
We provide a Jupyter environment that you can access and use to interactively write Python and .NET scripts. In our Jupyter environment, the SDKs come installed by default.

We provide the `lusidjam` library, which can be used to provide an authentication token without re-entering credentials into your Python scripts:

In [ ]:

config_loaders=[
    ArgsConfigurationLoader(access_token = RefreshingToken(), app_name = "LusidJupyterNotebook"),
    EnvironmentVariablesConfigurationLoader(),
    SecretsFileConfigurationLoader(secrets_path)
]

api_factory = lusid.extensions.SyncApiClientFactory(config_loaders=config_loaders)

Here we've built an api-factory using credentials stored in our Jupyter environment.

#### Luminesce cell magic

We also provide a custom magic command to query luminesce. The `%%luminesce` command is a magic command which passes the cell query string to Lumipy, which then returns a DataFrame.

We recommend using `%%luminesce` cell magic in FINBOURNE's JupyterHub. You can access this from the LUSID web app by navigating to **Additional Tools > Jupyter Notebooks** via the top left menu. Under the hood, when you run a cell with the `%%luminesce` command, JupyterHub passes the cell contents to the following function.

**Note**: If you're running a Notebook locally, you must [install Lumipy](https://support.lusid.com/v1/docs/how-do-i-use-lumipy-to-interact-with-luminesce) and run the following two code blocks to enable the cell contents to be passed to LumiPy. If you are using JupyterHub, you can skip straight to the first `%%luminesce` cell magic code block.

In [ ]:
# Only required if running locally:
!pip3 install -U lumipy

In [6]:
## Only required if running locally:
import os
from IPython.core.magic import (register_line_cell_magic)
from lumipy.client import Client
from lusidjam import RefreshingToken

token = RefreshingToken()
lumi_url = os.getenv("FBN_LUMI_API_URL")

@register_line_cell_magic
def luminesce(line, cell=None):
    query = cell if cell is not None else line

    try:
        lm_client = Client(token=token, api_url=lumi_url)
    except TypeError:
        # Attempt to use V2 SDK syntax if V1 syntax fails.
        # This gives V2 SDK support for luminesce magic
        lm_client = Client(access_token=token, api_url=lumi_url)

    df = lm_client.query_and_fetch(query)
            
    return df

# In an interactive session, we need to delete to avoid name conflicts for automagic to work on line magics.
del luminesce


In [ ]:
%%luminesce
SELECT * FROM Lusid.Instrument.Equity LIMIT 10

This cell magic runs any statement in the cell below the magic command, displaying a pandas dataframe containing the output of the Luminesce query.

In [ ]:
results = %luminesce SELECT * FROM Lusid.Instrument.Equity LIMIT 10
results.head()

This line magic runs the query on the same line after the magic command, allowing us to use the result of the query in our Python code.